## Market Beater
#### Kaggle's Hull Tactical - Market Prediction Competition
---

**Predicting Market Predictablity**

**By:** Dominic Smith (for I Understand AI)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from google.colab import drive

###LOAD DATA

In [ ]:

# =============================================================================
# Defining path to Google Drive data
FILE_PATH = '/content/drive/MyDrive/Kaggle Beat S&P'

# --- Loading S&P 500 Daily Price Data ---
try:
    sp500_df = pd.read_csv(FILE_PATH + '/SP500_daily.csv')
    print("S&P 500 data loaded successfully.")
except FileNotFoundError:
    print("S&P 500 file not found. Creating a placeholder DataFrame for demonstration.")
    date_range = pd.date_range(start='2015-01-01', end='2025-01-01', freq='B') # Business days
    placeholder_prices = 1500 + np.random.randn(len(date_range)).cumsum() * 5
    sp500_df = pd.DataFrame({'Date': date_range, 'Close': placeholder_prices})
    # Simulate holiday gaps
    holidays = np.random.choice(sp500_df.index, 20, replace=False)
    sp500_df.loc[holidays, 'Close'] = np.nan
    print("Placeholder data created with simulated holiday gaps.")

# --- Load Politician Trading Data ---
pol_df = pd.read_csv(FILE_PATH + '/insider1.csv')
print("Politician trading data loaded successfully.")

###DATA PREPROCESSING AND CLEANING

In [ ]:
# --- Load S&P 500 Price Data ---

try:
    # Correctly load 'SP500_daily.csv' file
    # and tell pandas to treat 'observation_date' as a date while loading
    sp500_df = pd.read_csv(
        FILE_PATH + '/SP500_daily.csv',
        parse_dates=['observation_date']
    )

    # Rename the columns to 'Date' and 'Close' so all the later code works
    sp500_df = sp500_df.rename(columns={
        'observation_date': 'Date',
        'SP500': 'Close'
    })

    print("Successfully loaded SP500_daily.csv and renamed columns.")
    print("DataFrame head:")
    print(sp500_df.head())

except FileNotFoundError:
    print("ERROR: Could not find 'SP500_daily.csv' in your Google Drive folder.")
    print("Please double-check the filename and folder location.")
    raise

In [ ]:
# --- 2. LOAD AND CLEAN POLITICIAN DATA (Revised with Explicit Type Conversion) ---
print("\n--- Step 2: Loading and Cleaning Politician Data ---")
try:
    pol_df = pd.read_csv(FILE_PATH + '/insider1.csv')

    # --- Convert 'Traded' column to datetime ---
    # Clean the string first, then convert.
    pol_df['Traded'] = pd.to_datetime(pol_df['Traded'].str.replace('\n', ' '), format='mixed', errors='coerce')

    # Add this line to clean the 'Published' date column
    pol_df['Published'] = pd.to_datetime(pol_df['Published'].str.replace('\n', ' '), format='mixed', errors='coerce')

    # --- Convert 'Price' column to numeric ---
    # The 'Price' column may have '$' signs. This will remove them and convert to a number.
    # 'coerce' will turn any errors (like 'N/A' text) into NaN (Not a Number).
    pol_df['Price'] = pd.to_numeric(pol_df['Price'].astype(str).str.replace('$', ''), errors='coerce')

    # --- Convert 'Size' column to a numeric estimate ---
    # This function handles ranges like '1K–15K'.
    def size_to_numeric(size_range):
        if isinstance(size_range, str):
            size_range = size_range.replace('K', '000').replace('M', '000000').replace('$', '')
            parts = size_range.split('–')
            if len(parts) == 2:
                try: return (float(parts[0]) + float(parts[1])) / 2
                except ValueError: return np.nan
        return np.nan

    pol_df['EstimatedValue'] = pol_df['Size'].apply(size_to_numeric)

    # Drop rows that couldn't be converted properly
    pol_df = pol_df.dropna(subset=['Traded', 'EstimatedValue', 'Price'])

    print("Politician data loaded and all relevant columns converted successfully.")
    print("\nCleaned pol_df dtypes:")
    print(pol_df[['Traded', 'Price', 'EstimatedValue']].info())
    print("-" * 50)

except FileNotFoundError:
    print("ERROR: Could not find the insider1.csv file.") # More specific error message
    raise

In [ ]:
# --- Process S&P 500 Data ---
# The 'observation_date' column is already the datetime index, so no need to convert or set index.

# Use forward-fill to handle holidays that might be present in the original data
sp500_df['Close'] = sp500_df['Close'].ffill()
print("\nHandled missing holiday data in original S&P 500 data using forward-fill.")

# --- Process Politician Trading Data ---
# Convert 'Traded' date column to datetime, coercing errors to NaT
pol_df['Traded'] = pd.to_datetime(pol_df['Traded'], errors='coerce')

# Function to convert trade size ranges (e.g., '1K–15K') to a numeric estimate
def size_to_numeric(size_range):
    if isinstance(size_range, str):
        size_range = size_range.replace('K', '000').replace('M', '000000').replace('$', '')
        parts = size_range.split('–')
        if len(parts) == 2:
            try:
                return (float(parts[0]) + float(parts[1])) / 2
            except ValueError:
                return np.nan
    return np.nan

pol_df['EstimatedValue'] = pol_df['Size'].apply(size_to_numeric)
pol_df = pol_df.dropna(subset=['EstimatedValue'])

###FEATURE ENGINEERING

In [ ]:
# Start with our clean S&P 500 data as the base
main_df = sp500_df.copy()

# Set the 'Date' column as the index and drop the original 'Date' column
main_df = main_df.set_index('Date')


# --- 1. Price-Based Features ---
# These are calculated first, using only the S&P 500 data
main_df['Volatility_20D'] = main_df['Close'].rolling(window=20).std()
main_df['Momentum_20D'] = main_df['Close'] - main_df['Close'].shift(20)
main_df['SMA_50'] = main_df['Close'].rolling(window=50).mean()

# --- 2. Political Trading Features ---
# Create a daily summary of political trades
# Ensure the index is a datetime type for proper merging
daily_trades = pol_df.groupby(pol_df['Traded'].dt.date).agg(
    buy_count=('Type', lambda x: (x == 'buy').sum()),
    sell_count=('Type', lambda x: (x == 'sell').sum()),
    buy_volume=('EstimatedValue', lambda x: x[pol_df['Type'] == 'buy'].sum()),
    sell_volume=('EstimatedValue', lambda x: x[pol_df['Type'] == 'sell'].sum())
)
daily_trades.index = pd.to_datetime(daily_trades.index) # Convert index to datetime

# Calculate net metrics
daily_trades['net_trades'] = daily_trades['buy_count'] - daily_trades['sell_count']
daily_trades['net_volume'] = daily_trades['buy_volume'] - daily_trades['sell_volume']

# --- 3. Merge Datasets ---
# Merge the political features into our main DataFrame.
# Use a 'left' merge to keep all the S&P 500 dates.
# Days with no political trades will have NaN values.
main_df = main_df.merge(
    daily_trades[['net_trades', 'net_volume']],
    left_index=True,
    right_index=True,
    how='left'
)

# Fill the NaN values for days with no trades with 0
main_df[['net_trades', 'net_volume']] = main_df[['net_trades', 'net_volume']].fillna(0)

# Reindex to a full daily calendar to ensure all weekdays are present *after* merging
# This step introduced NaT values, removing it.
# main_df = main_df.asfreq('D')

# Use forward-fill again to handle weekends and holidays introduced by asfreq
# main_df['Close'] = main_df['Close'].ffill()


# --- 4. Define the Target Variable ---
# The target is the 5-day future return. This must be the LAST step.
main_df['target'] = (main_df['Close'].shift(-5) - main_df['Close']) / main_df['Close']

print("Feature engineering and merge complete.")
print("DataFrame tail (showing recent data with political trades):")
print(main_df.tail(10))

###MODEL TRAINING

In [ ]:
# =============================================================================
# 5. MODEL TRAINING (with Hyperparameter Tuning v1)
# =============================================================================

# --- Prepare Final DataFrame ---
final_df = main_df.dropna()
print(f"Number of rows after dropna(): {len(final_df)}")

# --- Define Features (X) and Target (y) ---
# Using the original, best-performing feature set
features = ['Volatility_20D', 'Momentum_20D', 'SMA_50', 'net_trades', 'net_volume']
X = final_df[features]
y = final_df['target']

# --- Split Data Using a Specific Date ---
split_date = '2023-03-14'
first_test_date = final_df[final_df.index >= split_date].index.min()
split_date = first_test_date

X_train = X[X.index < split_date]
X_test = X[X.index >= split_date]
y_train = y[y.index < split_date]
y_test = y[y.index >= split_date]

print(f"\nTraining on {len(X_train)} samples (before {split_date.date()}).")
print(f"Testing on {len(X_test)} samples (on and after {split_date.date()}).")

# --- Train the XGBoost Model with Tuned Hyperparameters ---
# We are making the model more conservative to handle noisy financial data
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=2000,      # Increased estimators to allow for smaller learning rate
    learning_rate=0.01,     # <-- TUNED: Made smaller to learn more cautiously
    max_depth=3,            # <-- TUNED: Made shallower to prevent overfitting to noise
    subsample=0.7,
    colsample_bytree=0.7,
    n_jobs=-1,
    random_state=42,
    early_stopping_rounds=100 # Increased patience for the slower learning rate
)

model.fit(X_train, y_train,
          eval_set=[(X_test, y_test)],
          verbose=False)

print("\nModel training complete with tuned hyperparameters.")

###EVALUATION

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error by comparing the test set answers (y_test)
# with the model's predictions (y_pred)
mse = mean_squared_error(y_test, y_pred) # CORRECTED LINE

print(f"\nModel Performance on Test Set:")
print(f"Mean Squared Error (MSE): {mse:.6f}")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mse):.6f}")

# Plot predictions vs actuals from the test set
plt.figure(figsize=(15, 7))
plt.plot(y_test.index, y_test, label='Actual Future Returns', color='blue', alpha=0.7)
plt.plot(y_test.index, y_pred, label='Predicted Future Returns', color='red', linestyle='--')
plt.title('Model Predictions vs. Actual Values')
plt.xlabel('Date')
plt.ylabel('5-Day Return')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# =============================================================================
# ADVANCED EVALUATION METRICS
# =============================================================================
import numpy as np

# Create a new DataFrame to analyze results
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

# Determine the actual and predicted direction (1 for up, 0 for down)
results_df['Actual_Direction'] = np.where(results_df['Actual'] > 0, 1, 0)
results_df['Predicted_Direction'] = np.where(results_df['Predicted'] > 0, 1, 0)

# Calculate the directional accuracy
directional_accuracy = np.mean(results_df['Actual_Direction'] == results_df['Predicted_Direction'])

print(f"--- Model Performance Metrics ---")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mse):.6f}")
print(f"Directional Accuracy: {directional_accuracy:.2%}")

# See the first few predictions and their directions
print("\n--- Sample of Predictions vs Actuals ---")
print(results_df.head())

In [1]:
!git clone https://github.com/dominicftsmith/kaggle-market-prediction.git

Cloning into 'kaggle-market-prediction'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 30 (delta 12), reused 10 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 661.83 KiB | 3.15 MiB/s, done.
Resolving deltas: 100% (12/12), done.
